In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from train_unziplora_new_style import UnziploraAttentionProcessor, UnziploraAttentionWrapper, UNet2DConditionModel, UnziploraLinear

In [3]:
processor = UnziploraAttentionProcessor(4)

In [4]:
unet = UNet2DConditionModel()

<generator object Module.named_parameters at 0x298077e40>

In [10]:


for name, module in unet.named_modules():
    print(name, type(module).__name__)

    # break

 UNet2DConditionModel
conv_in Conv2d
time_proj Timesteps
time_embedding TimestepEmbedding
time_embedding.linear_1 Linear
time_embedding.act SiLU
time_embedding.linear_2 Linear
down_blocks ModuleList
down_blocks.0 CrossAttnDownBlock2D
down_blocks.0.attentions ModuleList
down_blocks.0.attentions.0 Transformer2DModel
down_blocks.0.attentions.0.norm GroupNorm
down_blocks.0.attentions.0.proj_in Conv2d
down_blocks.0.attentions.0.transformer_blocks ModuleList
down_blocks.0.attentions.0.transformer_blocks.0 BasicTransformerBlock
down_blocks.0.attentions.0.transformer_blocks.0.norm1 LayerNorm
down_blocks.0.attentions.0.transformer_blocks.0.attn1 Attention
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q Linear
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k Linear
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v Linear
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out ModuleList
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0 Linear
down_

In [ ]:
# from typing import Literal
import torch
# from train_unziplora_new_style import Attention, nn


from unziplora_unet import UnziploraUNet
from unziplora import UnziploraLinear
from accelerate import Accelerator

# TODO: take decomposed old code from server; don't do everything again


/home/dcor/benayakoren/miniconda3/envs/unziplora/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [45]:
unziplora = UnziploraUNet(4)
unziplora.unet

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): UnziploraLinear(
                  (linear): Linear(in_features=320, out_features=320, bias=False)
         

In [42]:
unziplora.save_unziplora("unziplora.pt" )

In [14]:
from diffusers import StableDiffusionXLPipeline
import torch
from pathlib import Path
from train_unziplora_old_style_new_unet import UnziploraUNet
from accelerate import Accelerator
with Path("../unziplora_weights/unziplora_weights").open("rb") as f:
    unziplora_weights = torch.load(f)

unziplora_weights
model_path = "stabilityai/stable-diffusion-xl-base-1.0"
unet_unziplora = UnziploraUNet(r=1, unet=model_path, unziplora_weights=unziplora_weights, accelerator=Accelerator(), p_remove=0.7)

# unet_unziplora = UnziploraUNet(r=1, unet=model_path, unziplora_weights=unziplora_weights, accelerator=Accelerator())
unet_unziplora.unet.requires_grad_(False)
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_path,
    unet=unet_unziplora.unet,
    
    # revision=self.args.revision,
    # torch_dtype=self.weight_dtype,
)
pipeline


/tmp/ipykernel_2505980/2647437713.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unziplora_weights = torch.load(f)


stabilityai/stable-diffusion-xl-base-1.0(remove:0.7):cpu


RuntimeError: Error(s) in loading state_dict for UnziploraLinear:
	Missing key(s) in state_dict: "_s_lora_column_mask", "_c_lora_column_mask", "linear.weight", "s_lora_A.weight", "s_lora_B.weight", "c_lora_A.weight", "c_lora_B.weight". 

In [33]:

from train_unziplora import get_args_parser
from common_typing import Args
from unziplora_trainer import UnziploraTrainer
from unziplora_inference import UnziploraInference

In [34]:
args = Args(validation_prompt="A cat and a dog")
args

Args(pretrained_model_name_or_path='stabilityai/stable-diffusion-xl-base-1.0', p_remove=0, pretrained_vae_model_name_or_path='madebyollin/sdxl-vae-fp16-fix', revision=None, dataset_name=None, dataset_config_name=None, instance_data_dir='/home/dcor/benayakoren/projects/unziplora/data/Styles/house_3d', cache_dir=None, image_column='image', caption_column=None, repeats=None, class_data_dir=None, instance_prompt='A [C] in the style of [S]', class_prompt=None, validation_prompt='A cat and a dog', custom_instance_prompts=None, num_validation_images=4, validation_epochs=50, with_prior_preservation=False, prior_loss_weight=1.0, num_class_images=100, gradient_accumulation_steps=1, mixed_precision=None, output_dir='output', logging_dir='log', seed=None, train_text_encoder=False, train_batch_size=1, sample_batch_size=1, num_train_epochs=1, max_train_steps=None, checkpointing_steps=500, checkpoints_total_limit=None, resume_from_checkpoint=None, gradient_checkpointing=False, learning_rate=0.0001, t

In [ ]:
inference_pipeline = UnziploraInference(args=args)
inference_pipeline

/home/dcor/benayakoren/miniconda3/envs/unziplora/lib/python3.12/site-packages/accelerate/accelerator.py:453: UserWarning: `log_with=tensorboard` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
01/27/2025 15:59:49 - DEBUG - my_logger - {'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'adam_weight_decay': 0.0001,
 'adam_weight_decay_text_encoder': 0.001,
 'allow_tf32': False,
 'cache_dir': None,
 'caption_column': None,
 'center_crop': True,
 'checkpointing_steps': 500,
 'checkpoints_total_limit': None,
 'class_data_dir': None,
 'class_prompt': None,
 'crops_coords_top_left_h': 0,
 'crops_coords_top_left_w': 0,
 'custom_instance_prompts': None,
 'dataloader_num_workers': 0,
 'dataset_config_name': None,
 'dataset_name': None,
 'enable_xformers_memory_efficient_attention': False,
 'gradient_accumulation_steps': 1,
 'gradient_checkpointing': False,
 'hub_m

stabilityai/stable-diffusion-xl-base-1.0(remove:0):cuda


{'dropout', 'attention_type', 'reverse_transformer_layers_per_block'} was not found in config. Values will be initialized to default values.


In [24]:
# dir(args)
# args.__dict__.keys()


parser = get_args_parser()
dir(parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_groups',
 '_actions',
 '_add_action',
 '_add_container_actions',
 '_check_conflict',
 '_check_value',
 '_defaults',
 '_get_args',
 '_get_formatter',
 '_get_handler',
 '_get_kwargs',
 '_get_nargs_pattern',
 '_get_option_tuples',
 '_get_optional_actions',
 '_get_optional_kwargs',
 '_get_positional_actions',
 '_get_positional_kwargs',
 '_get_value',
 '_get_values',
 '_handle_conflict_error',
 '_handle_conflict_resolve',
 '_has_negative_number_optionals',
 '_match_argument',
 '_match_arguments_partial',
 '_mutually_exclusive_groups',
 '_negative_number_matcher',
 '_option_string_actions',
 '_optional

In [25]:
parser.__dict__.keys()
# parser.register()

dict_keys(['description', 'argument_default', 'prefix_chars', 'conflict_handler', '_registries', '_actions', '_option_string_actions', '_action_groups', '_mutually_exclusive_groups', '_defaults', '_negative_number_matcher', '_has_negative_number_optionals', 'prog', 'usage', 'epilog', 'formatter_class', 'fromfile_prefix_chars', 'add_help', 'allow_abbrev', 'exit_on_error', '_positionals', '_optionals', '_subparsers'])

In [36]:
for name in parser._option_string_actions:
     name = name.lstrip("--") 
     if name not in args.__dict__.keys():
        print(name)

h
help


In [2]:
import torch
torch.load()